In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split


import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision.models import vgg16
#from torchvision.models import resnet101

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay

import cv2
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
import wandb
from copy import deepcopy
from tqdm import tqdm

import random
from IPython.display import clear_output



from fns4wandb import train_log, build_optimizer, set_lossfn

from functions import save2csv, save2json, read_in_json, import_imagedata, label_oh_tf, ImageProcessor
#from loop_fns import loop

import pickle
import csv
import json
from datetime import date

C:\ProgramData\anaconda3\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [3]:
model_vgg16 = vgg16(weights="IMAGENET1K_V1")
vgg_feats = model_vgg16.features
vgg_classifier = model_vgg16.classifier
print(vgg_classifier)
vgg_classifier.pop(6)
print(vgg_classifier)
#print(vgg_feats)
#print(model_vgg16)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)
Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
)


In [4]:
def metrics(label, prediction): #TypeError: Singleton array tensor(3) cannot be considered a valid collection.
    
    label= np.array(label.cpu())

    predictions_np = prediction.cpu().detach().numpy()
    #y_pred' parameter of f1_score must be an array-like or a sparse matrix. Got 7 instead.
    predicted_classes = np.argmax(predictions_np, axis=0)
    #print('metrics Label:   ', label)
    #print('metrics prediction   ', predicted_classes)
    #avg_f1_score = f1_score(label, predictions_np, average='macro')
    acc = accuracy_score(label, predicted_classes)
    
    return acc

In [5]:
"""for i in x_train[:3]:
    print(i[59:])
    print(len(i))
    #i=i[59:]
    #k = cv2.imread(i)
    #print(k, type(k))
    k = preprocess_im(i)
    
    k = k.squeeze()
    k = k.permute(1,2,0)
    k = np.array(k.cpu())*5
    plt.imshow(k)
    plt.show()"""

'for i in x_train[:3]:\n    print(i[59:])\n    print(len(i))\n    #i=i[59:]\n    #k = cv2.imread(i)\n    #print(k, type(k))\n    k = preprocess_im(i)\n    \n    k = k.squeeze()\n    k = k.permute(1,2,0)\n    k = np.array(k.cpu())*5\n    plt.imshow(k)\n    plt.show()'

In [6]:
wandb.login()

wandb: Currently logged in as: naughticalnonsence (antvis). Use `wandb login --relogin` to force relogin


True

In [7]:
print(os.getcwd())


#r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/'

C:\Users\nn268\Desktop


In [8]:
class Squeeze(nn.Module):
    def __init__(self):
        super(Squeeze, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        x = x.squeeze(0)
        return x

In [28]:
config = dict(
    epochs= 2, #30, 
    learning_rate =1.00E-05,
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 4e-5,
    ks = (3,5),
    loss_fn = 'CrossEntropy',
    scheduler=0.2,
    f_lin_lay = 4096, #7168 #1024*7 = 7168 4096
    random_seed = 1,
)

save_dict = {'Run' :'IDSW_VGG16_fine_280224_TEST',
            'Current_Epoch': 0,
            'save_location' : r'C:/Users/nn268/OneDrive - University of Sussex/Sussex/phd/antvis_exp1/saves/'}

col_dict =dict(
    pad = 5,
    colour = 'colour',
    size = [113,36]
)

"""
config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        #'dropout':{
        #    'values': [0.5, 0.4, 0.3]
        #},
        'epochs':{
            'value': 3
        },

        'first_lin_lay':{
            'values':[248832]
        },
        'optimizer': {
            'values': ['adam']
        },
        'learning_rate': {
                # a flat distribution between 0 and 0.1
                'distribution': 'log_uniform_values',
                'min': 1e-7,
                'max': 1e-2
            },
        'loss_fn': {
            'values': ['CrossEntropy', 'MSE'] #'MSE', 
        },
        'data_set':{
            'values':['Augmented']
        },
            'scheduler': {
            'values': [0.2, 0.3, 0.4, 0.6]
        },
        'ks': {
            'values': [(3,5)]
        },
        'channels':{
            'values': [3]
        },
        'num_classes': {
            'values': [11]
        },
        'model_name' : {'values': ['vgg16net_mlp']},
        'channels' : {'values': [3]},
        'image_path': {
            'values': [r'//smbhome.uscs.susx.ac.uk/nn268/Documents/PHD/antvis/optics/AugmentedDS_IDSW/']
        }
        }
    }
"""
#sweep_id = wandb.sweep(config, project=f"{run_title}")

"\nconfig = {\n    'method': 'random',\n    'metric':{\n        'goal': 'minimize',\n        'name': 'val_loss'},\n    'parameters': {\n        #'dropout':{\n        #    'values': [0.5, 0.4, 0.3]\n        #},\n        'epochs':{\n            'value': 3\n        },\n\n        'first_lin_lay':{\n            'values':[248832]\n        },\n        'optimizer': {\n            'values': ['adam']\n        },\n        'learning_rate': {\n                # a flat distribution between 0 and 0.1\n                'distribution': 'log_uniform_values',\n                'min': 1e-7,\n                'max': 1e-2\n            },\n        'loss_fn': {\n            'values': ['CrossEntropy', 'MSE'] #'MSE', \n        },\n        'data_set':{\n            'values':['Augmented']\n        },\n            'scheduler': {\n            'values': [0.2, 0.3, 0.4, 0.6]\n        },\n        'ks': {\n            'values': [(3,5)]\n        },\n        'channels':{\n            'values': [3]\n        },\n        'num_

In [32]:
def luminance(img):
    r,g,b = img[:,:,2], img[:,:,1], img[:,:,0]
    lum =(0.114*b)+(0.587*g)+(0.299*r)
    mean_lum = np.mean(lum)
    return mean_lum
    
"""
ValueError: could not broadcast input array from shape (224,224,3) into shape (0,224,3)
blank padding (144, 452, 3)
"""

def blank_padding(col_dict, img, size:tuple):

    img = cv2.resize(img, col_dict['size']) 
    w = size[1]
    h = size[0]

    delta_w = w -img.shape[1]
    delta_h = h-img.shape[0]

    half_delta_h = int(np.floor(delta_h/2))
    half_delta_w = int(np.floor(delta_w/2))
    
    avg_lum = int(luminance(img))
    new_x = np.full((h,w,3), avg_lum) 

    if img.shape[1]%2 ==0:
        if img.shape[0]%2 == 0:
            new_x[half_delta_h:-half_delta_h,half_delta_w:-half_delta_w,:] = img
        else:
            new_x[half_delta_h:-(half_delta_h+1),half_delta_w:-half_delta_w,:] = img
    else:
        if img.shape[0]%2 == 0:
            new_x[half_delta_h:-half_delta_h,half_delta_w:-(half_delta_w+1),:] = img #*#*#
        else:
            new_x[half_delta_h:-(half_delta_h+1),half_delta_w:-(half_delta_w+1),:] = img
            
    return new_x


def to_tensor(img):
    im_chan = img.shape[2]
    imgy, imgx = img.shape[0], img.shape[1]
    tensor = torch.tensor(img, dtype=torch.float32)
    tensor = F.normalize(tensor)
    tensor = tensor.permute(2,0,1)
    tensor = tensor.reshape(im_chan,imgy, imgx)
    return tensor
    
def preprocess_im(col_dict, img_path:str):
    img = cv2.imread(img_path) #
    if img is not None:
        #print('1',img.shape)
        img = blank_padding(col_dict, img, (224,224))
        #print('2',img.shape)
        img = to_tensor(img).to(device)
        #print(img.shape)
        return img
    else:
        return img


In [33]:

def loop(model, X, Y, loss_fn, device, col_dict, num_classes, pad_size =5, optimizer =None, scheduler= None, train =True):	# Train and Val loops. Default is train
    model = model
    total_samples = len(X)
    if train:
        model.train()
        #lr_ls = []
    else:
        model.eval()

    predict_list = []
    total_count = 0
    num_correct = 0
    current_loss = 0
    colour = col_dict['colour']
    size = col_dict['size']
    pad = col_dict['pad']
    issue_list =[]

    for idx, img in enumerate(X):
        #tense = tensoring(img).to(device)
        #prepro = ImageProcessor(device)
        
        tense = preprocess_im(col_dict, img_path= img) # col_dict, img_path:str
        if tense is None:
            issue_list.append((idx, img))
            continue
        else:

            prediction = model.forward(tense)
            label = label_oh_tf(Y[idx], num_classes).to(device)
            #if train:
            #	lr_ls.append(optimizer.param_groups[0]['lr'])
            loss = loss_fn(prediction, label)
            predict_list.append(prediction.argmax())

            if prediction.argmax() == label.argmax():
                num_correct +=1
                #if train:
                #	print(f'\n ########################### HIT ###########################  -- {idx} / {total_samples} \n')
            total_count+=1
            current_loss += loss.item()
            if train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                if scheduler:
                    scheduler.step()
        #print(num_correct/len(X))
    if train:
        return current_loss, predict_list, num_correct, model, optimizer, issue_list #, lr_ls
    else:
        return current_loss, predict_list, num_correct, issue_list



def test_loop(model, X, Y, loss_fn, device, col_dict,title, num_classes):
    model = model.eval()
    predict_list = []
    total_count =0
    num_correct = 0
    correct = 0
    colour = col_dict['colour']
    size = col_dict['size']

    with torch.no_grad():
        for idx, img in enumerate(X):
            #prepro = ImageProcessor(device)
            #tense = prepro.colour_size_tense(img, colour, size)
            tense = preprocess_im(img)
            prediction = model.forward(tense)
            label = label_oh_tf(Y[idx], num_classes).to(device)

            if prediction.argmax()==label.argmax():
                num_correct +=1
            total_count +=1
            correct +=(prediction.argmax()==label.argmax()).sum().item()

        acc = num_correct/total_count
        accuracy = 100*(acc)

        X = list(X)
        log_test_score(acc, accuracy, X)


In [36]:

def train_model(model,  x_train, x_val, y_train, y_val, config, best_acc=0): #train_dl, val_dl, 
    wandb.watch(model, log='all', log_freq=10)
    
    loss_fn = set_lossfn(config.loss_fn) # ****
    
    lr = config['learning_rate'] #1e-5 #config.learning_rate
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)#build_optimizer(model, optimizer=torch.optim.Adam(model.parameters(), lr=lr))#config.optimizer, config.learning_rate, config.weight_decay)
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=config.scheduler, last_epoch=-1) #gamma=config.scheduler, last_epoch=-1)
                                                                    #scheduler'
    ####
    
    #model = model.to(device)
    model.train()
    
    
    #losses= []
    #predictions = []
    t_loss_list = []
    v_loss_list = []
    t_predict_list = []
    v_predict_list = []
    t_accuracy_list = []
    v_accuracy_list = []
    t_label_list = []
    v_label_list = []
    #labels = []
    issue_list = []
    
    total_epochs = 0
    for epoch in tqdm(range(config['epochs'])): #config.epochs)):
        if epoch ==0:
            best_model=model
            best_acc=0
                                  
        t_loss, train_prediction, t_correct, model, optimizer, t_issue_list = loop(model=model, X=x_train, Y=y_train, loss_fn=loss_fn, device=device, col_dict=col_dict, num_classes=11, pad_size=col_dict['pad'], optimizer=optimizer, scheduler =scheduler)
        
        
        save_dict['t_loss_list'] = t_loss_list
        save_dict['t_labels'] = y_train
        save_dict['t_predict_list'] = t_predict_list 

        train_acc = (t_correct / len(x_train))
        t_accuracy_list.append(train_acc)
        
        save_dict['t_accuracy_list'] = t_accuracy_list 
        clear_output()
        
            
        print('validating...')
        
        v_loss, val_prediction, val_correct, v_issue_list = loop(model=model, X=x_val, Y=y_val, loss_fn=loss_fn, device=device, col_dict=col_dict, num_classes=11, pad_size=col_dict['pad'], train=False)


        save_dict['v_loss_list'] = v_loss_list
        save_dict['v_predict_list'] = v_predict_list  #
        save_dict['v_labels'] = y_val
        
        val_acc = (t_correct / len(x_val))
        v_accuracy_list.append(val_acc)
        save_dict['v_accuracy_list'] = v_accuracy_list  #
        
        issue_list.append([t_issue_list, v_issue_list])
        clear_output()
            
        total_epochs += epoch

        if val_acc > best_acc:

            best_acc = val_acc

            best_model = model#deepcopy(model)
            
            save_dict['Current_Epoch'] += config['epochs']
            save_dict['training_samples'] = len(x_train)
            save_dict['validation_samples'] = len(x_val)

            
            model_architecture = [nn.Sequential(
                PrintLayer(),
                vgg_feats,
                PrintLayer(),
                Flattern(),
                PrintLayer(),
                vgg_classifier,
                PrintLayer(),
                nn.Linear(4096,11),
                nn.Softmax(dim=0),
            )]

            

            title = save_dict['Run']
            save_location = save_dict['save_location']
            conf= dict(config)
            diction = {}
            d = date.today()
            d=str(d)
            diction.update({'Date':d})
            diction.update(save_dict)
            diction.update(conf)
            diction.update(col_dict)
            print(diction)
            
            save2json(conf, title, save_location)
            save2csv(diction, title, save_location)
            
            diction['model.state_dict'] = model.state_dict() #to('cpu').
            diction['model_architecture_untrained'] = model_architecture
            
            with open(f"{save_location}{title}.pkl", 'wb+') as f:
                pickle.dump(diction, f)
            
            
            
            print('improvment in metrics. model saved')

            #print('END Of SAve -----------------------------')
            #!nvidia-smi

        

        if (epoch+1)%2==0:
            train_log(t_loss, v_loss, epoch)
            wandb.log({'train_accuracy_%': train_acc, 'epoch':epoch})
            wandb.log({'val_accuracy_%': val_acc, 'epoch':epoch})
            
    model = best_model

    return model

class Flattern(nn.Module):
    def __init__(self):
        super(Flattern, self).__init__()
    def forward(self, x):
        #print(x.shape)
        x = x.flatten()
        return x

class PrintLayer(nn.Module):
    # usefule for printing network layers
    def __init__(self):
        super(PrintLayer, self).__init__()
    def forward(self, x):
        print(x.shape)
        return x


def pipeline(config, save_dict, col_dict): 
    # set model
    l = config['f_lin_lay']
    model_vgg16 = vgg16(weights="IMAGENET1K_V1")
    vgg_feats = model_vgg16.features
    vgg_classifier = model_vgg16.classifier
    print(vgg_classifier)
    vgg_classifier.pop(6)
    vgg = nn.Sequential(
        PrintLayer(),
        vgg_feats,
        PrintLayer(),
        Flattern(),
        #Squeeze(),
        #Flattern(),
        PrintLayer(),
        vgg_classifier,
        PrintLayer(),
        nn.Linear(l,11),
        nn.Softmax(dim=0),
        )
    
    #model_vgg16.to('cpu')
    
    vgg.to(device)
    
    # load in data
    file_path =  r'//smbhome.uscs.susx.ac.uk/nn268/Documents/PHD/antvis/optics/AugmentedDS_IDSW/'
    random_seed =1
    img_len = len(os.listdir(file_path))


    #print(ids[4])
    x, y = import_imagedata(file_path)

    x_train, x_test, y_train, y_tests = train_test_split(x,y, test_size=0.2, train_size=0.8,
                                     random_state=random_seed, shuffle=True)
    x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size=0.1, train_size=0.8,
                                     random_state=random_seed, shuffle=True)

    x_train, y_train = x_train[:10], y_train[:10] # for testing purposes, shortened data
    x_val, y_val = x_val[:10], y_val[:10]#
    #!nvidia-smi
    
    loss_list=[]
    #loss_fn = nn.CrossEntropyLoss()
    #loss_fn = nn.MSELoss()
    title=save_dict['Run']
    with wandb.init(project=title, config=config):
        config = wandb.config
        model = vgg

        model = train_model(model,x_train, x_val, y_train, y_val, config) #train_dl, val_dl

    return model

In [37]:
model = pipeline(config, save_dict, col_dict) #7,168

#def tr(config=None):
#    with wandb.init(config=config):
#        config = wandb.config
#        model, save_dict = pipeline(config)
        

#wandb.agent(sweep_id, tr, count=1)

100%|██████████| 2/2 [01:00<00:00, 30.48s/it]


epoch,▁▁▁
t_loss,▁
train_accuracy_%,▁
v_loss,▁
val_accuracy_%,▁
epoch,1
t_loss,24.06439
train_accuracy_%,0.0
v_loss,24.00764
val_accuracy_%,0.0


In [ ]:
Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)
Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
)


    vgg = nn.Sequential(
        PrintLayer(),
        vgg_feats,
        PrintLayer(),
        Flattern(),
        #Squeeze(),
        #Flattern(),
        PrintLayer(),
        vgg_classifier,
        PrintLayer(),
        nn.Linear(l,11),
        nn.Softmax(dim=0),
        )
    

In [ ]:
print(f"{save_dict['save_location']}{title}.pkl")

In [ ]:
    vgg = nn.Sequential(
        vgg_feats,
        Squeeze(),
        Flattern(),
        vgg_classifier,
        
        nn.Linear(4096,11),
        nn.Softmax(dim=0),
        )
#print(vgg)

print(vgg_feats)
print('------------------------------------')
print(vgg_classifier)

In [ ]:
!nvidia-smi

In [ ]:
# confusion matrix


def plot_confusion(predictions:list, actual:list, title:str):
    predict_list = [int(t.argmax()) for t in predictions]
    actual = [int(l.argmax()) for l in actual]

    actual = np.array(actual)
    predict_list = np.array(predict_list)


    #FixedLocator locations (3), usually from a call to set_ticks, does not match the number of labels (11).
    print(f'\n     {title}')
    train_epoch_matrix = confusion_matrix(actual, predict_list, labels= [0,1,2,3,4,5,6,7,8,9,10])
    disp= ConfusionMatrixDisplay(train_epoch_matrix, display_labels=[0,1,2,3,4,5,6,7,8,9,10])
    disp.plot()
    plt.show()


In [ ]:
t_predict = save_dict['t_predict_list']
t_labels = save_dict['t_labels']

v_predict = save_dict['v_predict_list'] # WHY IS THERE NOTHING IN V OREDICT LIST!
v_labels = save_dict['v_labels']

plot_confusion(t_predict, t_labels, 'Train Confusion Matrix')
plot_confusion(v_predict, v_labels, 'Validation Confusion Matrix')

In [ ]:
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        print(x)
        return x

In [ ]:
model =nn.Sequential(
    PrintLayer(),
    model_vgg16,
    PrintLayer(),
    Squeeze(),
    PrintLayer(),
    nn.Linear(4096,11),
    PrintLayer(),
    nn.Softmax(dim=0),
    PrintLayer()

).to(device)

In [ ]:

print(x_train[347])
print(x_train[348])
print(x_train[349])


p0 = x_train[348]
p_1 = x_train[347]
p1 = x_train[349]

p0 = cv2.imread(p0)
p_1 = cv2.imread(p_1)
p1 = cv2.imread(p1)

plt.imshow(p0)
p0 = cv2.resize(p0, [224, 72]) #img = cv2.resize(img, [224, 72])
print(p0.shape)
print(p_1.shape)
plt.show()
plt.imshow(p_1)
p_1 = cv2.resize(p_1, [224,72])
print(p_1.shape)
plt.show()
plt.imshow(p1)

plt.show()

In [ ]:

vgg = nn.Sequential(
        model_vgg16,
        Squeeze(),
        nn.Linear(4096,11),
        nn.Softmax(dim=0),
    )

#model_vgg16.to('cpu')

vgg.to(device)
next(vgg.parameters()).is_cuda

In [ ]:

x = preprocess_im(x_train[0])


train_prediction = model.forward(x)

In [ ]:
model.to('cpu').state_dict()